# Diffusion Model Implementation with U-Net on MNIST

---

## Author : Amir Atapour-Abarghouei, amir.atapour-abarghouei@durham.ac.uk

This notebook demonstrates the implementation of a diffusion model using a U-Net architecture on the MNIST dataset.

Copyright (c) 2024 Amir Atapour-Abarghouei, UK.

License : GNU General Public License Version 3

We are going to implement a very simple GAN. Let's start by importing what we need:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
from einops import rearrange
from torchvision import transforms

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Device is {device}!")

We should now line up the dataset we are going to use. We will be MNIST.

We'll load and preprocess the MNIST dataset, which contains 28x28 grayscale images of handwritten digits. We'll resize these images to 32x32 and convert them to 3 channels to match the input requirements of our U-Net model.

In [ ]:
# MNIST dataloader
train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST("./data", train=True, download=True,
                               transform=transforms.Compose([
                                   transforms.Resize((32, 32)),
                                   transforms.ToTensor(),
                                   transforms.Lambda(lambda x: x.repeat(3,1,1)),  # Convert 1-channel grayscale to 3 channels
                                   transforms.Normalize((0.5,), (0.5,))
                               ])), batch_size=16, shuffle=True)

# Check data loading
x, _ = next(iter(train_loader))
plt.imshow(rearrange(x[0], 'c h w -> h w c').cpu() * 0.5 + 0.5)
plt.show()

Now is time to define our model, a U-Net with DDPM.

We’ll define a U-Net model to serve as the backbone for our diffusion process. Note that this model, along with the rest of this implementation, is extremely simplified for educational purposes and shoud not be relied upon for any real applications.

In [ ]:
class UNet(nn.Module):
    def __init__(self, out_class=3):
        super().__init__()
        self.dconv_down1 = self.conv_block(72)
        self.dconv_down2 = self.conv_block(96)
        self.dconv_down3 = self.conv_block(128)
        self.dconv_down4 = self.conv_block(196)
        self.dconv_down5 = self.conv_block(256)
        self.bottleneck  = nn.LazyConv2d(256, 3,1,1, bias=False)
        self.dconv_up4 = self.conv_block(256, up=True)
        self.dconv_up3 = self.conv_block(196, up=True)
        self.dconv_up2 = self.conv_block(128, up=True)
        self.dconv_up1 = self.conv_block(96, up=True)
        self.conv_last = nn.LazyConv2d(out_class, 1)

    def conv_block(self, out_channels, up=False):
        ConvType = nn.LazyConvTranspose2d if up else nn.LazyConv2d
        return nn.Sequential(
            ConvType(out_channels, 4, 2, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU())

    def forward(self, x, c):
        start = x
        conv1 = self.dconv_down1(x)     # skip-connection 1
        conv2 = self.dconv_down2(conv1) # skip-connection 2
        conv3 = self.dconv_down3(conv2) # skip-connection 3
        conv4 = self.dconv_down4(conv3) # skip-connection 4

        # simple inject of the conditionals into the network
        ch = F.one_hot(c, num_classes=10).float().to(device).view(-1,10,1,1)

        x = self.bottleneck(conv4)
        x = torch.cat([x, conv4, ch.repeat(1,1,x.size(2),x.size(3))], dim=1) # injection 1
        x = self.dconv_up4(x)
        x = torch.cat([x, conv3, ch.repeat(1,1,x.size(2),x.size(3))], dim=1) # injection 2
        x = self.dconv_up3(x)
        x = torch.cat([x, conv2], dim=1)
        x = self.dconv_up2(x)
        x = torch.cat([x, conv1], dim=1)
        x = self.dconv_up1(x)
        x = torch.cat([x, start], dim=1)
        return self.conv_last(x)

print(f"The architecture has been created!")

Now, we will define the DDPM model using the U-Net architecture and set up the training loop.

In [ ]:
class DDPM(nn.Module):
    def __init__(self,):
        super(DDPM, self).__init__()
        self.net = UNet()

    # Algorithm 1 in DDPM paper with conditionals
    def forward(self, x, c):
        _ts = torch.randint(1, T+1, (x.shape[0],)).to(x.device)
        eps = torch.randn_like(x)
        x_t = (sqrtab[_ts] * x + sqrtmab[_ts] * eps)
        return F.mse_loss(eps, self.net(x_t, c))

    # Algorithm 2 in DDPM paper with torch.arange(n_sample) conditionals
    def sample(self, n_sample, size):
        x_i = torch.randn(n_sample, *size).to(device)
        for i in range(T, 0, -1):
            z = torch.randn(n_sample, *size).to(device) if i > 1 else 0
            eps = self.net(x_i, torch.arange(n_sample))
            x_i = (oneover_sqrta[i] * (x_i - eps * mab_over_sqrtmab_inv[i]) + sigma_t[i] * z )
        return x_i

    print(f"The DDPM model has been created!")

Here’s the training loop, where we optimise the model to predict the noise added to the images.

In [ ]:
# Training parameters
T = 400
beta1 = 1e-4
beta2 = 0.02
beta_t = (beta2 - beta1) * torch.arange(0, T + 1, dtype=torch.float32, device=device) / T + beta1
alpha_t = 1 - beta_t
log_alpha_t = torch.log(alpha_t)
alphabar_t = torch.cumsum(log_alpha_t, dim=0).exp()
sqrtab = torch.sqrt(alphabar_t).view(-1,1,1,1)
sqrtmab = torch.sqrt(1 - alphabar_t).view(-1,1,1,1)
oneover_sqrta = 1 / torch.sqrt(alpha_t)
mab_over_sqrtmab_inv = (1 - alpha_t) / sqrtmab.squeeze()
sigma_t = torch.sqrt(beta_t)

ddpm = DDPM().to(device)
optim = torch.optim.Adam(ddpm.parameters(), lr=1e-4)

# Training loop
for step in range(20000):
    x, c = next(iter(train_loader))
    x, c = x.to(device), c.to(device)

    # Forward pass
    loss = ddpm(x, c)

    # Backward pass
    optim.zero_grad()
    loss.backward()
    optim.step()

    # Print progress
    if step % 500 == 0:
        print(f"Step {step}, Loss: {loss.item():.4f}")


Finally, we will generate new images using the trained diffusion model by sampling from the reverse process.

In [ ]:
# Sampling new images
ddpm.eval()
with torch.no_grad():
    xh = ddpm.sample(10, (3, 32, 32))
    img_grid = rearrange(xh*0.5+0.5, '(b1 b2) c h w -> (b1 h) (b2 w) c', b1=1)
    plt.imshow((img_grid.cpu()*255).int().numpy())
    plt.show()